# Named Entity Recognition with Spacy

We have used tutorial from: https://towardsdatascience.com/train-ner-with-custom-training-data-using-spacy-525ce748fab7

### Configuration

In [1]:
from __future__ import unicode_literals, print_function
from collections import defaultdict
import plac
import random
from pathlib import Path
import spacy
from tqdm import tqdm

### Training

In [2]:
TRAIN_DATA = [
    ('My main research interests are in machine learning, artificial intelligence, and theoretical computer science.', {
        'entities': [(34,50, 'AREA'), (52, 75, 'AREA'), (81, 109, 'AREA')]
    }),
    ('My primary research areas are computational Biology, Bioinformatics and Machine learning.', {
        'entities': [(53,67, 'AREA'), (72,88, 'AREA')]
    }),
    ('I am interested in the intersection of machine learning and systems.', {
        'entities': [(39,55, 'AREA'), (60,77, 'AREA')]
    }),
    ('I work on developing and using Machine Learning, AI, and Data Science methods.', {
        'entities': [(31,47, 'AREA'), (49,51, 'AREA'), (57,69, 'AREA')]
    }),
    ('I am interested in the intersection of machine learning and systems.', {
        'entities': [(39,55, 'AREA'), (60,67, 'AREA')]
    }),
    ('Research Area CS Education', {
    'entities': [(14,26, 'AREA')]
    }),
    ('Research Area CS Education', {
    'entities': [(14,26, 'AREA')]
    }),
('''Research Interests Computational complexity, communication complexity, coding theory, algorithms, and combinatorics''', {
		'entities': [(18, 43, 'AREA'), (45, 69, 'AREA'), (71, 84, 'AREA'), (86, 96, 'AREA'), (102, 115, 'AREA'), ]
}),
('''Research Interests: Computer security, formal methods, automated reasoning''', {
		'entities': [(20, 37, 'AREA'), (39, 53, 'AREA'), (55, 74, 'AREA'), ]
}),
('''Research Interests: Computational Fluid Dynamics, Hydrodynamic Instabilities, Turbulence, Ocean Modeling''', {
		'entities': [(20, 48, 'AREA'), (50, 76, 'AREA'), (78, 88, 'AREA'), (90, 104, 'AREA'), ]
}),
('''Research Interests Signal Processing for wireless communications Computational statistics Statistical signal processing Information theory Bioinformatics''', {
		'entities': [(19, 36, 'AREA'), (65, 89, 'AREA'), (90, 119, 'AREA'), (120, 138, 'AREA'), (139, 153, 'AREA'), ]
}),
('''Research Interests Electronic Design Automation (EDA)Layout optimization Logic synthesis Programmable devices and timing optimization Bridging optimization''', {
		'entities': [(19, 47, 'AREA'), (54, 72, 'AREA'), (73, 88, 'AREA'), (89, 109, 'AREA'), (114, 133, 'AREA'), ]
}),
('''Research interests: Distributed computing; Analysis of algorithms; Data structures; Computational geometry; Graph algorithms''', {
		'entities': [(20, 41, 'AREA'), (43, 65, 'AREA'), (67, 82, 'AREA'), (84, 106, 'AREA'), (108, 124, 'AREA'), ]
}),
('''My research interests are in the general areas of Computer Networking, Information Security, and Distributed Systems.''', {
		'entities': [(50, 69, 'AREA'), (71, 91, 'AREA'), (97, 116, 'AREA'), ]
}),
('''Research Interests Computer Graphics HCI Multimedia Systems Scientific Visualization''', {
		'entities': [(19, 36, 'AREA'), (37, 40, 'AREA'), (41, 59, 'AREA'), ]
}),
('''Research Interests Open source computing, scientometrics, digital libraries.''', {
		'entities': [(19, 40, 'AREA'), (42, 56, 'AREA'), (58, 75, 'AREA'), ]
}),
('''Research Interests: Software Engineering; Software Testing; Software Security;''', {
		'entities': [(20, 40, 'AREA'), (42, 58, 'AREA'), (60, 77, 'AREA')]
}),
('''My own interests are in Software Engineering of Large Systems, Open Source Software, new and innovative Computer Programming languages.''', {
		'entities': [(25, 44, 'AREA'), (63, 83, 'AREA'), (93, 134, 'AREA'), ]
}),
('''Areas of Interest Neuromorphic Computing Machine Learning Image and Pattern Recognition Education''', {
		'entities': [(18, 40, 'AREA'), (41, 57, 'AREA'), (68, 87, 'AREA'), ]
}),
('''His research interests and activities are in real-time and safety-critical system design, scheduling theory, resource allocation and sharing in distributed computing environments, and algorithm design and analysis.''', {
		'entities': [(75, 88, 'AREA'), (144, 178, 'AREA'), (184, 200, 'AREA'), (205, 213, 'AREA'), ]
}),
('''Jonathan's research interests center around computer networking and computer security''', {
		'entities': [(44, 62, 'AREA'), (68, 84, 'AREA'), ]
}),
('''Research Interests: Computer security, network security''', {
		'entities': [(20, 37, 'AREA'), (39, 55, 'AREA'), ]
}),
('''His primary research interests are database systems, object-oriented systems and software engineering.''', {
		'entities': [(35, 51, 'AREA'), (53, 76, 'AREA'), (81, 101, 'AREA'), ]
}),
('''Research Interests My major areas of research are in computational fluid dynamics, adaptive methods for the numerical solution of pdes in complex geometries, and large-scale parallel computing.''', {
		'entities': [(53, 81, 'AREA'), (83, 99, 'AREA'), (162, 192, 'AREA'), ]
}),
('''Research interests: Hidden web search and data integration; information systems design; database design; knowledge representation.''', {
		'entities': [(20, 37, 'AREA'), (60, 86, 'AREA'), (88, 103, 'AREA'), (105, 129, 'AREA'), ]
}),
]

### Faculty dataset

In [3]:
# There are 6525 total number of faculty files
faculty_dataset = defaultdict(str)
for i in range(6525):
    path = "./data/compiled_bios/" + str(i) +".txt"
    f = open(path, "r")    
    for sen in f:
        faculty_dataset[i] = sen.strip()

In [4]:
len(faculty_dataset)

6525

### Setup Language Model

In [5]:
model = None
output_dir=Path("./")
n_iter=200

#load the model
if model is not None:
    nlp = spacy.load(model)  
    print("Loaded model '%s'" % model)
else:
    nlp = spacy.blank('en')  
    print("Created blank 'en' model")
    
#set up the pipeline
if 'ner' not in nlp.pipe_names:
    ner = nlp.create_pipe('ner')
    nlp.add_pipe(ner, last=True)
else:
    ner = nlp.get_pipe('ner')

Created blank 'en' model


### Training

In [6]:
for _, annotations in TRAIN_DATA:
    for ent in annotations.get('entities'):
        ner.add_label(ent[2])

other_pipes = [pipe for pipe in nlp.pipe_names if pipe != 'ner']
with nlp.disable_pipes(*other_pipes):  # only train NER
    optimizer = nlp.begin_training()
    for itn in range(n_iter):
        random.shuffle(TRAIN_DATA)
        losses = {}
        for text, annotations in tqdm(TRAIN_DATA):
            nlp.update(
                [text],  
                [annotations],  
                drop=0.5,  
                sgd=optimizer,
                losses=losses)
        print(losses)

 20%|██        | 5/25 [00:00<00:00, 35.70it/s]

{'ner': 174.74414750328287}


 16%|█▌        | 4/25 [00:00<00:00, 32.60it/s]

{'ner': 117.4512726413538}


 12%|█▏        | 3/25 [00:00<00:00, 29.21it/s]

...

{'ner': 5.559146353145739}


 12%|█▏        | 3/25 [00:00<00:00, 27.28it/s]

{'ner': 2.294263845962511}


100%|██████████| 25/25 [00:01<00:00, 24.62it/s]

{'ner': 3.506706678258857}


### Testing (Test with training data)

In [7]:
for text, _ in TRAIN_DATA:
    doc = nlp(text)
    print('Entities', [(ent.text, ent.label_) for ent in doc.ents])

Entities [('Distributed computing', 'AREA'), ('Analysis of algorithms', 'AREA'), ('Data structures', 'AREA'), ('Computational geometry', 'AREA'), ('Graph algorithms', 'AREA')]
Entities [('computer networking', 'AREA'), ('computer security', 'AREA')]
Entities [('CS Education', 'AREA')]
Entities [('system design', 'AREA'), ('distributed computing environments', 'AREA'), ('algorithm design', 'AREA'), ('analysis', 'AREA')]
Entities [('Computer security', 'AREA'), ('formal methods', 'AREA'), ('automated reasoning', 'AREA')]
Entities [('Hidden web search', 'AREA'), ('information systems design', 'AREA'), ('database design', 'AREA'), ('knowledge representation', 'AREA')]
Entities [('computational fluid dynamics', 'AREA'), ('adaptive methods', 'AREA'), ('large-scale parallel computing', 'AREA')]
Entities [('Computer security', 'AREA'), ('network security', 'AREA')]
Entities [('Bioinformatics', 'AREA'), ('Machine learning', 'AREA')]
Entities [('Computational Fluid Dynamics', 'AREA'), ('Hydrodyn

### Testing (Test with all data)

1. In every document, we first find a sentence that start includes "research interest, research interests, interested in"
2. We run our model on this sentence

In [8]:
matcher = spacy.matcher.Matcher(nlp.vocab)
# Add match ID "HelloWorld" with no callback and one pattern
matcher.add("research interest", None,
            [{"LOWER": "research"}, {"LOWER": "interest"}],
            [{"LOWER": "research"}, {"LOWER": "interests"}],
            [{"LOWER": "interested"}, {"LOWER": "in"}],
            [{"LOWER": "research"}, {"LOWER": "areas"}],
            [{"LOWER": "research"}, {"LOWER": "area"}],
            [{"LOWER": "research"}, {"LOWER": "addresses"}],
           )

In [9]:
result = defaultdict(str)

In [10]:
for i, text in faculty_dataset.items():

    doc = nlp(text)
    matches = matcher(doc)
    
    if matches:
        for match_id, start, end in matches:
            span = doc[start:start+30]  # The matched span
            break
    else:
        result[i] = ''
        continue
    
    doc2 = nlp(span.text)
    entities = [(ent.text, ent.label_) for ent in doc2.ents]
    if entities:
        result[i] = entities[0][0]
    else:
        result[i] = ''

### Save result

In [11]:
import os,codecs,re

def save_result(dic_data, out_path):

    with codecs.open(out_path,'w',encoding='utf-8',errors='ignore') as f:
        for i, data in dic_data.items():
            if data:
                f.write(data+'\n')
            else:
                f.write('\n')

save_result(result,'./data/areas')